In [3]:
# pip install pillow pillow-heif   # run once in your venv

import os
from pathlib import Path
from PIL import Image, ImageOps
from pillow_heif import register_heif_opener

register_heif_opener()

def convert_heic_to_webp(
    root_folder,
    quality=80,
    method=6,
    inplace=True,
    overwrite=False,
    delete_original=False,   # 👈 new: remove HEIC after a successful WEBP save
):
    root = Path(root_folder).expanduser().resolve()
    heic_exts = {".heic", ".heif", ".heics", ".heifs"}

    total = converted = skipped = failed = 0

    for src in root.rglob("*"):
        if not (src.is_file() and src.suffix.lower() in heic_exts):
            continue
        total += 1

        dst = (
            src.with_suffix(".webp")
            if inplace
            else (root / "webp_output" / src.relative_to(root)).with_suffix(".webp")
        )

        if dst.exists() and not overwrite:
            print(f"⏭️  Skip (already converted): {dst}")
            skipped += 1
            continue

        try:
            with Image.open(src) as im:
                im = ImageOps.exif_transpose(im)
                if im.mode not in ("RGB", "RGBA"):
                    im = im.convert("RGBA" if "A" in im.getbands() else "RGB")
                icc = im.info.get("icc_profile")

                dst.parent.mkdir(parents=True, exist_ok=True)
                im.save(
                    dst,
                    format="WEBP",
                    quality=quality,
                    method=method,
                    icc_profile=icc,
                    lossless=False,
                    optimize=True,  # optional: a bit smaller files
                )

            # Remove original HEIC only if conversion succeeded and target exists
            if delete_original and dst.exists():
                try:
                    src.unlink()
                    print(f"🗑️  Deleted original: {src}")
                except Exception as de:
                    print(f"⚠️  Couldn't delete original {src}: {de}")

            print(f"✅ {src} -> {dst}")
            converted += 1

        except Exception as e:
            print(f"❌ Failed: {src}\n   {e}")
            failed += 1

    print("\n—— Summary ——")
    print(f"Found HEICs: {total}")
    print(f"Converted:   {converted}")
    print(f"Skipped:     {skipped}")
    print(f"Failed:      {failed}")


In [4]:
convert_heic_to_webp(
    "images/shelter/honeydew",
    quality=80,
    method=6,
    inplace=True,        # write .webp next to the source
    overwrite=False,     # keep existing .webp if present
    delete_original=True # 👈 remove the old HEICs after convert
)


🗑️  Deleted original: /Users/77wu/Documents/GitHub/Blog2.0/images/shelter/honeydew/IMG_4727.HEIC
✅ /Users/77wu/Documents/GitHub/Blog2.0/images/shelter/honeydew/IMG_4727.HEIC -> /Users/77wu/Documents/GitHub/Blog2.0/images/shelter/honeydew/IMG_4727.webp
🗑️  Deleted original: /Users/77wu/Documents/GitHub/Blog2.0/images/shelter/honeydew/IMG_4746.HEIC
✅ /Users/77wu/Documents/GitHub/Blog2.0/images/shelter/honeydew/IMG_4746.HEIC -> /Users/77wu/Documents/GitHub/Blog2.0/images/shelter/honeydew/IMG_4746.webp
🗑️  Deleted original: /Users/77wu/Documents/GitHub/Blog2.0/images/shelter/honeydew/IMG_4741.HEIC
✅ /Users/77wu/Documents/GitHub/Blog2.0/images/shelter/honeydew/IMG_4741.HEIC -> /Users/77wu/Documents/GitHub/Blog2.0/images/shelter/honeydew/IMG_4741.webp
🗑️  Deleted original: /Users/77wu/Documents/GitHub/Blog2.0/images/shelter/honeydew/IMG_4734.HEIC
✅ /Users/77wu/Documents/GitHub/Blog2.0/images/shelter/honeydew/IMG_4734.HEIC -> /Users/77wu/Documents/GitHub/Blog2.0/images/shelter/honeydew/IMG_47